### **Laboratorio 5**

- Juan Pablo Solis
- Isabella Miralles

In [6]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

train_df.head()


Train shape: (7613, 5)
Test shape: (3263, 4)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
nan_percentage = train_df[['keyword', 'location']].isna().mean() * 100
print(nan_percentage)


keyword      0.801261
location    33.272035
dtype: float64


In [8]:
train_df = train_df.drop(columns=['location'])
test_df = test_df.drop(columns=['location'])
train_df.head()


,id,keyword,text,target
0,1,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Se decidio eliminar la columna location debido a que tenia un 33% de NaN, en comparacion a Keyword el cual tiene solo un 0.8%.

#### **Preprocesamiento de datos**

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    
    text = text.lower()
    
    
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    
    text = re.sub(r'[@
    
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    
    text = re.sub(r'\d+', '', text)
    
    
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text


clean_train = train_df.copy()
clean_test = test_df.copy()

clean_train['text'] = clean_train['text'].apply(clean_text)
clean_test['text'] = clean_test['text'].apply(clean_text)


clean_train = clean_train[['keyword', 'text', 'target']]
clean_test = clean_test[['keyword', 'text']]



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  keyword                                               text  target
0     NaN       deeds reason earthquake may allah forgive us       1
1     NaN              forest fire near la ronge sask canada       1
2     NaN  residents asked shelter place notified officer...       1
3     NaN  people receive wildfires evacuation orders cal...       1
4     NaN  got sent photo ruby alaska smoke wildfires pou...       1
  keyword                                               text
0     NaN                        happened terrible car crash
1     NaN  heard earthquake different cities stay safe ev...
2     NaN  forest fire spot pond geese fleeing across str...
3     NaN              apocalypse lighting spokane wildfires
4     NaN                typhoon soudelor kills china taiwan


In [14]:
clean_test.head()

,keyword,text
0,NaN,happened terrible car crash
1,NaN,heard earthquake different cities stay safe ev...
2,NaN,forest fire spot pond geese fleeing across str...
3,NaN,apocalypse lighting spokane wildfires
4,NaN,typhoon soudelor kills china taiwan
